In [ ]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 4.8 MB/s eta 0:00:00


In [ ]:
# classification(comments)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from transformers import Trainer, TrainingArguments, pipeline, AutoTokenizer, AutoModelForMaskedLM
from sklearn.model_selection import train_test_split
import torch
import pandas as pd
from datasets import Dataset

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/NLP/Dataset/darija_clean.csv')

In [ ]:
df1=df.copy()
df2=df.copy()

In [ ]:
df1

,cleaned_text,label,sentence_length
0,الي كيقولوا الشرع اعطاه اربعة وهو محامي عاد وم...,positive,967
1,الي كيقولوا الشرع اعطاه اربعة وهو محامي عاد وم...,neutral,967
2,واش هادا كايبغيك باله عليك لي كايهدر معاك بهاد...,negative,993
3,فظائح ومهازل والشعب التونسي يعاني البطالة والف...,Negative,983
4,يعطيه الصحة على الأقل لقينا واحد فاهم الحكاية ...,Positive,883
...,...,...,...
43693,Ya ta7an,Negative,8
43694,أنا أؤمن,neutral,8
43695,كيدايرة؟,Neutral,8
43696,اول لايك,neutral,8


In [ ]:
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Assuming `df1` is your initial dataset
# Split into 80% training and 20% testing
train_df, test_df = train_test_split(df1, test_size=0.2, random_state=42)

# Further split the training set into 65% training and 15% validation
train_df, val_df = train_test_split(train_df, test_size=0.15, random_state=42)

# Create Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")
print(f"Test set size: {len(test_dataset)}")

Training set size: 29714
Validation set size: 5244
Test set size: 8740


In [ ]:
from transformers import AutoTokenizer

# Load the tokenizer for DarijaBERT
tokenizer = AutoTokenizer.from_pretrained("SI2M-Lab/DarijaBERT")

# Define the tokenization function
def tokenize_function(examples):
    return tokenizer(examples["cleaned_text"], padding="max_length", truncation=True, max_length=512)

# Apply tokenization to all datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/307 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/779 [00:00<?, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


vocab.txt:   0%|          | 0.00/879k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.


Map:   0%|          | 0/29714 [00:00<?, ? examples/s]

Map:   0%|          | 0/5244 [00:00<?, ? examples/s]

Map:   0%|          | 0/8740 [00:00<?, ? examples/s]

In [ ]:
# Normalize labels
def normalize_label(label):
    label_mapping = {
        'negative': 'negative',
        'Negative': 'negative',
        'négatif': 'negative',
        'neutral': 'neutral',
        'Neutral': 'neutral',
        'positif': 'positive',
        'Positive': 'positive',
        'positive': 'positive',
        'mixed': 'mixed',
        'Mixed': 'mixed',
        'neutre': 'neutral'
    }
    return label_mapping.get(label.lower(), label)

# Map labels to numeric values
label_map = {'negative': 0, 'neutral': 1, 'positive': 2, 'mixed': 3}

# Apply normalization and mapping
train_dataset = train_dataset.map(lambda x: {'label': normalize_label(x['label'])})
val_dataset = val_dataset.map(lambda x: {'label': normalize_label(x['label'])})
test_dataset = test_dataset.map(lambda x: {'label': normalize_label(x['label'])})

train_dataset = train_dataset.map(lambda x: {'labels': label_map[x['label']]})
val_dataset = val_dataset.map(lambda x: {'labels': label_map[x['label']]})
test_dataset = test_dataset.map(lambda x: {'labels': label_map[x['label']]})

Map:   0%|          | 0/29714 [00:00<?, ? examples/s]

Map:   0%|          | 0/5244 [00:00<?, ? examples/s]

Map:   0%|          | 0/8740 [00:00<?, ? examples/s]

Map:   0%|          | 0/29714 [00:00<?, ? examples/s]

Map:   0%|          | 0/5244 [00:00<?, ? examples/s]

Map:   0%|          | 0/8740 [00:00<?, ? examples/s]

In [ ]:
print(f"Unique labels in the train dataset: {set(train_dataset['labels'])}")
print(f"Unique labels in the validation dataset: {set(val_dataset['labels'])}")

Unique labels in the train dataset: {0, 1, 2, 3}
Unique labels in the validation dataset: {0, 1, 2, 3}


In [ ]:
# Set format for PyTorch tensors
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
val_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [ ]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType
import evaluate
import numpy as np

# Load the model for sequence classification
model = AutoModelForSequenceClassification.from_pretrained("SI2M-Lab/DarijaBERT", num_labels=4)

# Configure LoRA for Sequence Classification
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,  # Sequence Classification
    inference_mode=False,
    r=8,  # Low-rank dimension
    lora_alpha=32,
    lora_dropout=0.1,
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Define metrics
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=-1)

    accuracy_metric = evaluate.load("accuracy")
    f1_metric = evaluate.load("f1")

    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")

    return {
        "accuracy": accuracy["accuracy"],
        "f1": f1["f1"]
    }

training_args = TrainingArguments(
    output_dir="./lora_finetuned_model",
    num_train_epochs=1,
    per_device_train_batch_size=8,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=50,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to=["none"],  # Avoid using wandb or other integrations
    save_total_limit=1,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

# Train the model
trainer.train()

The `xla_device` argument has been deprecated in v4.4.0 of Transformers. It is ignored and you can safely remove it from your `config.json` file.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at SI2M-Lab/DarijaBERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-16-5a044979dea0>:52: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
# Save the fine-tuned model
model.save_pretrained("lora_finetuned_darijaBERT")

In [ ]:
import json
import matplotlib.pyplot as plt

# Path to the training log file
log_file_path = "./logs/trainer_state.json"

# Read and parse the log file
with open(log_file_path, 'r') as log_file:
    logs = json.load(log_file)

# Extract relevant data
steps = []
train_loss = []
eval_accuracy = []
eval_f1 = []

for log_entry in logs.get("log_history", []):
    if "loss" in log_entry:
        steps.append(log_entry.get("step", 0))
        train_loss.append(log_entry["loss"])
    if "eval_accuracy" in log_entry:
        eval_accuracy.append(log_entry["eval_accuracy"])
    if "eval_f1" in log_entry:
        eval_f1.append(log_entry["eval_f1"])

# Plot the metrics
plt.figure(figsize=(15, 5))

# Loss curve
plt.subplot(1, 3, 1)
plt.plot(steps, train_loss, label="Train Loss", color="blue")
plt.xlabel("Steps")
plt.ylabel("Loss")
plt.title("Training Loss")
plt.legend()

# Accuracy curve
plt.subplot(1, 3, 2)
plt.plot(steps[:len(eval_accuracy)], eval_accuracy, label="Eval Accuracy", color="green")
plt.xlabel("Steps")
plt.ylabel("Accuracy")
plt.title("Evaluation Accuracy")
plt.legend()

# F1-score curve
plt.subplot(1, 3, 3)
plt.plot(steps[:len(eval_f1)], eval_f1, label="Eval F1", color="orange")
plt.xlabel("Steps")
plt.ylabel("F1-Score")
plt.title("Evaluation F1-Score")
plt.legend()

plt.tight_layout()
plt.show()

In [ ]:
# Evaluate on test set
test_results = trainer.evaluate(test_dataset)
print("Test Results:", test_results)